In [1]:
import gym
import torch
import pickle

from model import DeepQNetwork, Agent
import numpy as np
import time
from gym import wrappers

In [2]:
env = gym.make('SpaceInvaders-v0')
brain = Agent(gamma=0.95, epsilon=1.0,
              alpha=0.003, maxMemorySize=1000,
              replace=None)

brain.Q_eval.state_dict(torch.load("/home/plicca/Documents/code/atari/model/qeval"))
brain.Q_next.state_dict(torch.load("/home/plicca/Documents/code/atari/model/qnext"))
brain.memory = pickle.load(open('/home/plicca/Documents/code/atari/model/mem.npy', 'rb'))

print("Brain loaded")

Brain loaded


In [8]:
scores = []
epsHistory = []
numGames = 50
batch_size = 16
done = False
env = gym.make('SpaceInvaders-v0')

In [ ]:
for i in range(numGames):
    print('starting game ', i + 1, 'epsilon: %.4f' % brain.EPSILON)
    epsHistory.append(brain.EPSILON)
    observation = env.reset()
    frames = [np.sum(observation[15:200,30:125], axis=2)]
    score = 0
    lastAction = 0
    delta = time.clock()
    done = False
    while not done:
        if len(frames) == 3:
            action = brain.chooseAction(frames)
            frames = []
            #current_time = time.clock()
            #print(current_time - delta)
            #delta = current_time
        else:
            action = lastAction
        #env.render()
        observation_, reward, done, info = env.step(action)
        score += reward
        frames.append(np.sum(observation_[15:200,30:125], axis=2))
        if done and info['ale.lives'] == 0:
            reward = -100
        brain.storeTransition(np.mean(observation[15:200,30:125], axis=2), action, reward,
                              np.mean(observation_[15:200,30:125], axis=2))
        observation = observation_

        brain.learn(batch_size)
        lastAction = action
    print(score, ':::', time.clock() - delta)

starting game  1 epsilon: 0.0500


/home/plicca/miniconda3/envs/atari/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  
/home/plicca/miniconda3/envs/atari/lib/python3.7/site-packages/ipykernel_launcher.py:31: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead


480.0 ::: 71.91035100000227
starting game  2 epsilon: 0.0500
205.0 ::: 38.39864499999749
starting game  3 epsilon: 0.0500
225.0 ::: 63.877502999999706
starting game  4 epsilon: 0.0500
300.0 ::: 43.21649500000058
starting game  5 epsilon: 0.0500
175.0 ::: 36.20476200000121
starting game  6 epsilon: 0.0500
800.0 ::: 115.60747199999969
starting game  7 epsilon: 0.0500
150.0 ::: 36.022892999997566
starting game  8 epsilon: 0.0500
255.0 ::: 69.11289100000067
starting game  9 epsilon: 0.0500
155.0 ::: 37.82827600000019
starting game  10 epsilon: 0.0500
155.0 ::: 40.028487999999925
starting game  11 epsilon: 0.0500


In [ ]:
torch.save(brain.Q_eval.state_dict(), "/home/plicca/Documents/code/atari/model/qeval")
torch.save(brain.Q_next.state_dict(), "/home/plicca/Documents/code/atari/model/qnext")
pickle.dump(brain.memory, open("/home/plicca/Documents/code/atari/model/mem.npy", 'wb'))
env.close()